In [235]:
import torch
from torch import nn,optim
from torch.autograd.variable import Variable
from torchvision import transforms,datasets
from tensorboardX import SummaryWriter
import torchvision.utils as vutils
from torch.distributions.normal import Normal

writer = SummaryWriter('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/log/')

In [236]:
#torch.device()
cuda=torch.device('cuda:1')

In [237]:
def mnist():
    compose=transforms.Compose([transforms.ToTensor()])#,transforms.Normalize((0.5,),(0.5,))])
    outp_dir='/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets'
    return datasets.MNIST(root=outp_dir,train=True,transform=compose,download=True)

In [238]:
mnist_data=mnist()

In [239]:
dataloader=torch.utils.data.DataLoader(mnist_data,batch_size=128,shuffle=True)

In [223]:
for a,b in enumerate(dataloader):
    break

In [224]:
b[0].shape

torch.Size([128, 1, 28, 28])

In [240]:
class disc(torch.nn.Module):
    def __init__(self):
        super(disc,self).__init__()
        inp_dim=784
        out_dim=1
        #self.l1=nn.Sequential(nn.Linear(784,1024),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l2=nn.Sequential(nn.Linear(784,512),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l3=nn.Sequential(nn.Linear(512,256),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.out=nn.Sequential(nn.Linear(256,1),nn.Sigmoid())
    def forward(self,x):
        #x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        x=self.out(x)
        return x

In [241]:
discriminator=disc().to(cuda)

In [244]:
class gen(torch.nn.Module):
    def __init__(self):
        super(gen,self).__init__()
        self.l1=nn.Sequential(nn.Linear(100,256),nn.LeakyReLU(0.2))#,nn.Dropout(0.5))
        #self.bn1=nn.BatchNorm1d(256)
        self.l2=nn.Sequential(nn.Linear(256,512),nn.LeakyReLU(0.2))#,nn.Dropout(0.5))
        #self.bn2=nn.BatchNorm1d(512)
        self.l3=nn.Sequential(nn.Linear(512,784),nn.Tanh())#nn.LeakyReLU(0.2))#,nn.Dropout(0.5))
        #self.out=nn.Sequential(nn.Linear(1024,784),nn.Tanh())
    def forward(self,x):
        x=self.l1(x)
        #x=self.bn1(x)
        x=self.l2(x)
        #x=self.bn2(x)
        x=self.l3(x)
        #x=self.out(x)
        return x

In [245]:
generator=gen().to(cuda)

In [246]:
def vec_img(vec):
    return vec.view(vec.size(0),1,28,28)
def img_vec(img):
    return img.view(img.shape[0],784)

In [247]:
def ones(batch_size):
    return Variable(torch.ones(batch_size,1))
def zeros(batch_size):
    return Variable(torch.zeros(batch_size,1))

In [248]:
g_optim=optim.Adam(generator.parameters(),lr=0.0002,betas=(0.5,0.999))
d_optim=optim.Adam(discriminator.parameters(),lr=0.0002,betas=(0.5,0.999))

In [249]:
loss=nn.BCELoss()

In [ ]:
num_epochs=200
count=0
for i in range(num_epochs):
    for n_batch,(real_batch,labels) in enumerate(dataloader):
        real_data=real_batch.to(cuda)
        batch_size=real_batch.shape[0]
        
        #####train the discriminator
        d_optim.zero_grad()
        #backprop fake data with labels
        
        fake_img=generator(Variable(Normal(torch.zeros(batch_size,100),torch.ones(batch_size,100)).sample().to(cuda)))#torch.randn(batch_size,100)).to(cuda))
        pred_fake_labels=discriminator(fake_img)
        error11=loss(pred_fake_labels,
                     torch.from_numpy(np.random.randint(10, 30, size=(batch_size,1)).astype("float") / 100).float().to(cuda))
        #zeros(real_batch.shape[0]).to(cuda))
        #error11.backward()
        #backprop with real data
        
        pred_real_labels=discriminator(img_vec(real_data).to(cuda))
        error12=loss(pred_real_labels,
                     torch.from_numpy(np.random.randint(80, 90, size=(batch_size,1)).astype("float") / 100).float().to(cuda)
                    )#ones(real_batch.shape[0]).to(cuda))
        #error12.backward()
        errord=error11+error12
        errord.backward()
        d_optim.step()
        ####train the generator
        
        
        inp=Variable(Normal(torch.zeros(batch_size,100),torch.ones(batch_size,100)).sample().to(cuda))#Variable(torch.randn(batch_size,100)).to(cuda)
        fake_image=generator(inp)
        fake_pred_label=discriminator(fake_image)
        error2=loss(fake_pred_label,
                    torch.from_numpy(np.random.randint(80, 90, size=(batch_size,1)).astype("float") / 100).float().to(cuda))
        #ones(batch_size).to(cuda))
        error2.backward()
        g_optim.step()
        print("epoch:{} step:{} g_loss:{} d_loss:{}".format(i,n_batch,error2,error11+error12))
        if count%10==0:
            writer.add_image("real_image",vutils.make_grid(real_data, padding=2, normalize=True), count)
            writer.add_image("gen_images",vutils.make_grid(fake_image.view(fake_image.shape[0],1,28,28), padding=2,
                                                           normalize=True), count)
            #writer.add_image("input_images",vutils.make_grid(inp, padding=2, normalize=True), count)
        count=count+1

epoch:0 step:0 g_loss:0.7066501975059509 d_loss:1.3753306865692139
epoch:0 step:1 g_loss:0.706118106842041 d_loss:1.349776268005371
epoch:0 step:2 g_loss:0.7059217691421509 d_loss:1.323810338973999
epoch:0 step:3 g_loss:0.703411877155304 d_loss:1.2943774461746216
epoch:0 step:4 g_loss:0.6979173421859741 d_loss:1.2823498249053955
epoch:0 step:5 g_loss:0.6889536380767822 d_loss:1.2634994983673096
epoch:0 step:6 g_loss:0.6782522201538086 d_loss:1.2515400648117065
epoch:0 step:7 g_loss:0.6678253412246704 d_loss:1.2462302446365356
epoch:0 step:8 g_loss:0.6548946499824524 d_loss:1.2385270595550537
epoch:0 step:9 g_loss:0.6448488235473633 d_loss:1.238755226135254
epoch:0 step:10 g_loss:0.6402683854103088 d_loss:1.2391245365142822
epoch:0 step:11 g_loss:0.6417575478553772 d_loss:1.2389591932296753
epoch:0 step:12 g_loss:0.6502582430839539 d_loss:1.229560375213623
epoch:0 step:13 g_loss:0.6561918258666992 d_loss:1.2072374820709229
epoch:0 step:14 g_loss:0.6658383011817932 d_loss:1.1982059478759

epoch:0 step:121 g_loss:1.4313080310821533 d_loss:0.9713256359100342
epoch:0 step:122 g_loss:1.6583826541900635 d_loss:0.9719774127006531
epoch:0 step:123 g_loss:1.3667118549346924 d_loss:0.970920205116272
epoch:0 step:124 g_loss:1.7440686225891113 d_loss:0.9669190049171448
epoch:0 step:125 g_loss:1.5323927402496338 d_loss:0.9505748152732849
epoch:0 step:126 g_loss:1.3621383905410767 d_loss:0.9567327499389648
epoch:0 step:127 g_loss:1.9471789598464966 d_loss:0.988268256187439
epoch:0 step:128 g_loss:1.408968210220337 d_loss:1.0008418560028076
epoch:0 step:129 g_loss:1.336464524269104 d_loss:0.973438024520874
epoch:0 step:130 g_loss:1.6466898918151855 d_loss:0.9612661600112915
epoch:0 step:131 g_loss:1.4589678049087524 d_loss:0.9610103964805603
epoch:0 step:132 g_loss:1.3670934438705444 d_loss:0.9794391393661499
epoch:0 step:133 g_loss:1.617112398147583 d_loss:0.9624850749969482
epoch:0 step:134 g_loss:1.4100518226623535 d_loss:0.9673997163772583
epoch:0 step:135 g_loss:1.49892473220825

epoch:0 step:241 g_loss:1.3422298431396484 d_loss:0.9706887006759644
epoch:0 step:242 g_loss:1.3112719058990479 d_loss:0.9710356593132019
epoch:0 step:243 g_loss:1.4487990140914917 d_loss:0.9616453647613525
epoch:0 step:244 g_loss:1.4764838218688965 d_loss:0.9588065147399902
epoch:0 step:245 g_loss:1.3900412321090698 d_loss:0.950772762298584
epoch:0 step:246 g_loss:1.338800311088562 d_loss:0.9679156541824341
epoch:0 step:247 g_loss:1.3759369850158691 d_loss:0.9541712403297424
epoch:0 step:248 g_loss:1.4069457054138184 d_loss:0.9543060660362244
epoch:0 step:249 g_loss:1.2949233055114746 d_loss:0.9524558186531067
epoch:0 step:250 g_loss:1.5246448516845703 d_loss:0.9736558198928833
epoch:0 step:251 g_loss:1.4758695363998413 d_loss:0.9669225811958313
epoch:0 step:252 g_loss:1.305567741394043 d_loss:0.943917989730835
epoch:0 step:253 g_loss:1.3766944408416748 d_loss:0.9514229893684387
epoch:0 step:254 g_loss:1.4103431701660156 d_loss:0.9612476825714111
epoch:0 step:255 g_loss:1.414537191390

epoch:0 step:361 g_loss:1.4164416790008545 d_loss:0.9327061772346497
epoch:0 step:362 g_loss:1.392082929611206 d_loss:0.933340847492218
epoch:0 step:363 g_loss:1.5163092613220215 d_loss:0.9358485341072083
epoch:0 step:364 g_loss:1.416534662246704 d_loss:0.9446483850479126
epoch:0 step:365 g_loss:1.4020004272460938 d_loss:0.9466190338134766
epoch:0 step:366 g_loss:1.4559638500213623 d_loss:0.9479621052742004
epoch:0 step:367 g_loss:1.4246264696121216 d_loss:0.9403586983680725
epoch:0 step:368 g_loss:1.3362743854522705 d_loss:0.9441710710525513
epoch:0 step:369 g_loss:1.4060373306274414 d_loss:0.954250693321228
epoch:0 step:370 g_loss:1.3003551959991455 d_loss:0.9635459184646606
epoch:0 step:371 g_loss:1.423600196838379 d_loss:0.9535917043685913
epoch:0 step:372 g_loss:1.433899164199829 d_loss:0.937803328037262
epoch:0 step:373 g_loss:1.3832988739013672 d_loss:0.9386545419692993
epoch:0 step:374 g_loss:1.47494637966156 d_loss:0.9271242618560791
epoch:0 step:375 g_loss:1.3737614154815674 

epoch:1 step:12 g_loss:1.3642315864562988 d_loss:0.9496302604675293
epoch:1 step:13 g_loss:1.4194004535675049 d_loss:0.9317456483840942
epoch:1 step:14 g_loss:1.5021355152130127 d_loss:0.9377675652503967
epoch:1 step:15 g_loss:1.3656657934188843 d_loss:0.9464446306228638
epoch:1 step:16 g_loss:1.435098648071289 d_loss:0.9347135424613953
epoch:1 step:17 g_loss:1.4342010021209717 d_loss:0.947758138179779
epoch:1 step:18 g_loss:1.4182395935058594 d_loss:0.9302563071250916
epoch:1 step:19 g_loss:1.386892557144165 d_loss:0.9382498264312744
epoch:1 step:20 g_loss:1.4205222129821777 d_loss:0.9415880441665649
epoch:1 step:21 g_loss:1.467814326286316 d_loss:0.9148328304290771
epoch:1 step:22 g_loss:1.3497387170791626 d_loss:0.9569872617721558
epoch:1 step:23 g_loss:1.3497366905212402 d_loss:0.949840247631073
epoch:1 step:24 g_loss:1.452881932258606 d_loss:0.9301537275314331
epoch:1 step:25 g_loss:1.4535984992980957 d_loss:0.934861958026886
epoch:1 step:26 g_loss:1.4386694431304932 d_loss:0.9298

...

epoch:106 step:197 g_loss:1.4142606258392334 d_loss:0.931695818901062
epoch:106 step:198 g_loss:1.4145478010177612 d_loss:0.9311059713363647
epoch:106 step:199 g_loss:1.419123888015747 d_loss:0.9257253408432007
epoch:106 step:200 g_loss:1.3669159412384033 d_loss:0.9471452832221985
epoch:106 step:201 g_loss:1.4069201946258545 d_loss:0.9172601699829102
epoch:106 step:202 g_loss:1.41965651512146 d_loss:0.9257219433784485
epoch:106 step:203 g_loss:1.3662832975387573 d_loss:0.9420114159584045
epoch:106 step:204 g_loss:1.4150581359863281 d_loss:0.9215787649154663
epoch:106 step:205 g_loss:1.4264490604400635 d_loss:0.9314247965812683
epoch:106 step:206 g_loss:1.3901007175445557 d_loss:0.9338629245758057
epoch:106 step:207 g_loss:1.4118835926055908 d_loss:0.9312103986740112
epoch:106 step:208 g_loss:1.401944875717163 d_loss:0.9366209506988525
epoch:106 step:209 g_loss:1.437660574913025 d_loss:0.9128512144088745
epoch:106 step:210 g_loss:1.4600536823272705 d_loss:0.9100431203842163
epoch:106 st

epoch:106 step:314 g_loss:1.407322883605957 d_loss:0.9366110563278198
epoch:106 step:315 g_loss:1.4215617179870605 d_loss:0.9157751798629761
epoch:106 step:316 g_loss:1.4401949644088745 d_loss:0.9184487462043762
epoch:106 step:317 g_loss:1.4437828063964844 d_loss:0.9196293354034424
epoch:106 step:318 g_loss:1.4310702085494995 d_loss:0.9148112535476685
epoch:106 step:319 g_loss:1.44736647605896 d_loss:0.9146591424942017
epoch:106 step:320 g_loss:1.4039705991744995 d_loss:0.9284099340438843
epoch:106 step:321 g_loss:1.3825536966323853 d_loss:0.9281090497970581
epoch:106 step:322 g_loss:1.429455280303955 d_loss:0.9190280437469482
epoch:106 step:323 g_loss:1.451871633529663 d_loss:0.932720422744751
epoch:106 step:324 g_loss:1.3941683769226074 d_loss:0.9246348142623901
epoch:106 step:325 g_loss:1.3860187530517578 d_loss:0.9319366216659546
epoch:106 step:326 g_loss:1.400488257408142 d_loss:0.9273325204849243
epoch:106 step:327 g_loss:1.4584758281707764 d_loss:0.9170980453491211
epoch:106 ste

epoch:106 step:434 g_loss:1.3971247673034668 d_loss:0.9315618276596069
epoch:106 step:435 g_loss:1.3986456394195557 d_loss:0.93062424659729
epoch:106 step:436 g_loss:1.4338445663452148 d_loss:0.909564197063446
epoch:106 step:437 g_loss:1.436856985092163 d_loss:0.9335334300994873
epoch:106 step:438 g_loss:1.406562328338623 d_loss:0.923521876335144
epoch:106 step:439 g_loss:1.4315500259399414 d_loss:0.9261445999145508
epoch:106 step:440 g_loss:1.4411652088165283 d_loss:0.9247333407402039
epoch:106 step:441 g_loss:1.4296436309814453 d_loss:0.9287039041519165
epoch:106 step:442 g_loss:1.4423933029174805 d_loss:0.9196454286575317
epoch:106 step:443 g_loss:1.3876310586929321 d_loss:0.9384269714355469
epoch:106 step:444 g_loss:1.410508155822754 d_loss:0.929855465888977
epoch:106 step:445 g_loss:1.3947478532791138 d_loss:0.925580620765686
epoch:106 step:446 g_loss:1.4051631689071655 d_loss:0.9254504442214966
epoch:106 step:447 g_loss:1.4283303022384644 d_loss:0.9208450317382812
epoch:106 step:

epoch:107 step:86 g_loss:1.3816001415252686 d_loss:0.926859974861145
epoch:107 step:87 g_loss:1.394524097442627 d_loss:0.9317259788513184
epoch:107 step:88 g_loss:1.4336044788360596 d_loss:0.9274340867996216
epoch:107 step:89 g_loss:1.4235751628875732 d_loss:0.9174327850341797
epoch:107 step:90 g_loss:1.432856798171997 d_loss:0.9257410168647766
epoch:107 step:91 g_loss:1.4170657396316528 d_loss:0.9227839708328247
epoch:107 step:92 g_loss:1.4035887718200684 d_loss:0.9189584255218506
epoch:107 step:93 g_loss:1.3951377868652344 d_loss:0.936622142791748
epoch:107 step:94 g_loss:1.4158875942230225 d_loss:0.920843780040741
epoch:107 step:95 g_loss:1.4290480613708496 d_loss:0.9178459644317627
epoch:107 step:96 g_loss:1.4187445640563965 d_loss:0.9327110648155212
epoch:107 step:97 g_loss:1.4191389083862305 d_loss:0.927946925163269
epoch:107 step:98 g_loss:1.4746123552322388 d_loss:0.9240130186080933
epoch:107 step:99 g_loss:1.4684295654296875 d_loss:0.911665141582489
epoch:107 step:100 g_loss:1

epoch:107 step:205 g_loss:1.4313957691192627 d_loss:0.9253261089324951
epoch:107 step:206 g_loss:1.3949270248413086 d_loss:0.9297036528587341
epoch:107 step:207 g_loss:1.4018372297286987 d_loss:0.9252086281776428
epoch:107 step:208 g_loss:1.4130384922027588 d_loss:0.9410228729248047
epoch:107 step:209 g_loss:1.410888671875 d_loss:0.9304181337356567
epoch:107 step:210 g_loss:1.3967783451080322 d_loss:0.9377085566520691
epoch:107 step:211 g_loss:1.379817247390747 d_loss:0.9216685891151428
epoch:107 step:212 g_loss:1.4380590915679932 d_loss:0.9349068403244019
epoch:107 step:213 g_loss:1.452418565750122 d_loss:0.9262661933898926
epoch:107 step:214 g_loss:1.4256579875946045 d_loss:0.9290580749511719
epoch:107 step:215 g_loss:1.41141939163208 d_loss:0.9245283603668213
epoch:107 step:216 g_loss:1.3889596462249756 d_loss:0.9451642036437988
epoch:107 step:217 g_loss:1.4254581928253174 d_loss:0.9206350445747375
epoch:107 step:218 g_loss:1.3850088119506836 d_loss:0.9413998126983643
epoch:107 step

In [ ]:
#real_batch.shape
#img_vec(real_data).shape
!nvidia-smi

In [213]:
#import matplotlib.pyplot as plt
#plt.imshow( real_batch.permute(2, 3, 1, 0)[:,:,0,127] )


In [217]:
#Normal(torch.zeros(100,1),torch.ones(100,1)).sample()

tensor([[ 2.5039],
        [ 0.0123],
        [ 1.4642],
        [ 0.9987],
        [ 0.6696],
        [-1.6128],
        [ 0.1617],
        [ 0.7186],
        [ 0.5012],
        [ 1.2506],
        [-0.0359],
        [ 0.2739],
        [-0.1060],
        [-1.7490],
        [ 0.5973],
        [-0.8484],
        [ 0.4738],
        [ 0.9884],
        [-1.6503],
        [-0.5792],
        [-0.6551],
        [-0.7969],
        [-0.6223],
        [-0.0912],
        [-0.2350],
        [-0.0477],
        [-0.3462],
        [ 0.5517],
        [ 0.0556],
        [ 0.7046],
        [-0.2974],
        [ 0.6332],
        [-1.4388],
        [ 1.6708],
        [ 0.0683],
        [-2.0739],
        [ 1.4443],
        [-0.0145],
        [ 0.1280],
        [-0.9243],
        [ 1.5951],
        [ 0.5199],
        [-1.1882],
        [ 1.5522],
        [ 1.4854],
        [ 0.7088],
        [ 1.4371],
        [-0.0131],
        [-0.5829],
        [-0.3736],
        [-0.4483],
        [ 1.0794],
        [ 0.

In [18]:
#torch.cuda.device_count()

In [46]:
fake=generator(Variable(torch.randn(128,100)).to(cuda))

In [47]:
fake.shape

torch.Size([128, 784])

In [141]:
fake_image.shape

torch.Size([128, 784])

In [48]:
fake=fake.view(fake.shape[0],1,28,28)

In [49]:
fake.shape

torch.Size([128, 1, 28, 28])

In [60]:
fake=fake.permute(2,3,1,0)

In [61]:
fake.shape

torch.Size([28, 28, 1, 128])

In [62]:
fake=fake.detach()

In [26]:
import matplotlib.pyplot as plt
plt.imshow(fake[:,:,0,0].cpu())

NameError: name 'fake' is not defined

In [54]:
fake[:,:,0,0].shape

torch.Size([28, 28])

In [48]:
torch.rand(128, 25000)

tensor([[0.1996, 0.3229, 0.3850,  ..., 0.9994, 0.4935, 0.4944],
        [0.6967, 0.9485, 0.7779,  ..., 0.6393, 0.5998, 0.7679],
        [0.7908, 0.7462, 0.0635,  ..., 0.3390, 0.1744, 0.5888],
        ...,
        [0.6379, 0.1173, 0.7387,  ..., 0.4279, 0.0489, 0.6575],
        [0.0479, 0.1617, 0.4118,  ..., 0.7797, 0.9842, 0.3863],
        [0.9835, 0.0140, 0.9695,  ..., 0.0496, 0.1219, 0.9448]])

In [53]:
import numpy as np
np.random.rand(1,2)

array([[0.30827648, 0.06284637]])

In [55]:
x=np.random.randint(90, 120, size=(50,1)).astype("float") / 100

In [57]:
torch.from_numpy(x)

tensor([[1.0400],
        [1.0300],
        [0.9400],
        [1.0800],
        [1.0500],
        [1.0100],
        [1.0600],
        [1.0000],
        [0.9400],
        [1.0300],
        [0.9100],
        [1.1400],
        [0.9000],
        [1.1700],
        [0.9900],
        [1.0600],
        [1.1100],
        [0.9900],
        [1.1800],
        [1.0500],
        [1.0400],
        [1.0500],
        [1.0300],
        [1.0800],
        [1.1300],
        [1.1200],
        [1.0600],
        [1.0900],
        [1.1700],
        [1.0600],
        [0.9600],
        [0.9700],
        [1.1100],
        [1.0500],
        [0.9800],
        [1.0600],
        [0.9800],
        [0.9700],
        [0.9000],
        [0.9200],
        [1.1200],
        [0.9300],
        [1.1700],
        [1.0000],
        [1.1600],
        [1.1200],
        [1.0600],
        [1.0800],
        [1.1800],
        [0.9000]], dtype=torch.float64)

In [2]:
!nvidia-smi

Tue Dec 24 03:35:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P6            On   | 00000000:17:00.0 Off |                    0 |
| N/A   38C    P0    46W /  90W |  14231MiB / 15261MiB |     87%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P6            On   | 00000000:45:00.0 Off |                    0 |
| N/A   40C    P0    29W /  90W |  13200MiB / 15261MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T